In [ ]:
required <- c('ggplot2', 'plyr', 'dplyr',
              'caret', 'reshape2', 'ROCR', 'e1071', 'randomForest', 'gbm',
              'car', 'missForest', 'ggmap', 'lubridate', 'Amelia', 'magrittr',
              'doMC', 'pROC')
installed <- rownames(installed.packages())
tobeinstalled <- required[!(required %in% installed)]
print(paste('Installing', tobeinstalled))
install.packages(tobeinstalled)

In [ ]:
install.packages('bartMachine_1.2.1.tar.gz')

In [1]:
source('common.R')

Loading required package: Rcpp
## 
## Amelia II: Multiple Imputation
## (Version 1.7.4, built: 2015-12-05)
## Copyright (C) 2005-2016 James Honaker, Gary King and Matthew Blackwell
## Refer to http://gking.harvard.edu/amelia/ for more information
## 

Attaching package: ‘plyr’

The following object is masked from ‘package:lubridate’:

    here



## Load data

In [2]:
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [3]:
trainset.labels <- read.csv('trainset_labels.csv', na.strings = "")
statusGroups <- dummyVars(~ status_group, data = trainset.labels)
trainset.labels <- cbind(trainset.labels, data.frame(predict(statusGroups, trainset.labels)))

trainset.values <- read.csv('trainset_values.csv', na.strings = "") %>%
  prepareDataSet('train')

testset.values <- read.csv('testset_values.csv', na.strings = "") %>% 
  prepareDataSet('test')

In [4]:
head(trainset.values)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,ellip.h,source,source_type,source_class,waterpoint_type,waterpoint_type_group,date_recorded_year,date_recorded_year_ago,date_recorded_month,date_recorded_day,type
1,69572,6000,2011-03-14,Roman,1390,Roman,34.93809,-9.856322,none,0,⋯,spring,spring,groundwater,communal standpipe,communal standpipe,2011,4,3,14,train
2,8776,0,2013-03-06,Grumeti,1399,GRUMETI,34.69877,-2.147466,Zahanati,0,⋯,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,2,3,6,train
3,34310,25,2013-02-25,Lottery Club,686,World vision,37.46066,-3.821329,Kwa Mahundi,0,⋯,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,2,2,25,train
4,67743,0,2013-01-28,Unicef,263,UNICEF,38.48616,-11.1553,Zahanati Ya Nanyumbu,0,⋯,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,2,1,28,train
5,19728,0,2011-07-13,Action In A,0,Artisan,31.13085,-1.825359,Shuleni,0,⋯,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,4,7,13,train
6,9944,20,2011-03-13,Mkinga Distric Coun,0,DWE,39.1728,-4.765587,Tajiri,0,⋯,other,other,unknown,communal standpipe multiple,communal standpipe,2011,4,3,13,train


## Machine Learning

In [5]:
#reducedTrainset <- head(join(trainset.values, trainset.labels), 10000)
reducedTrainset <- join(trainset.values, trainset.labels)
inTrain <- createDataPartition(y=reducedTrainset$status_group, p=0.8, list=FALSE)
training <- reducedTrainset[inTrain,]
testing <- reducedTrainset[-inTrain,]

nzvs <- nzv(training)
predictors <- colnames(training)[!(colnames(training) %in% c('id', 'status_group', 'recorded_by', 'type','wpt_name',
                                                             'ward', 'lga',
                                                             'scheme_name', 'public_meeting', 'gps_height', 'num_private',
                                                             'installer', 'subvillage', 'funder', 'permit')) &
                                   !grepl('status_group', colnames(training))]


Joining by: id


In [56]:
nmin <- sum(training$status_group == 'functional needs repair')
others <- training$status_group != 'functional needs repair'
undersampled <- rbind(training[others,][sample.int(sum(others), nmin),],
                      training[training$status_group == 'functional needs repair',])

table(undersampled$status_group)


             functional functional needs repair          non functional 
                   2030                    3454                    1424 

In [6]:
options(java.parameters="-Xmx55g")
library(bartMachine)
set_bart_machine_num_cores(3)

Loading required package: rJava
Loading required package: car
Loading required package: randomForest
randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: missForest
Loading required package: foreach
Loading required package: itertools
Loading required package: iterators
Welcome to bartMachine v1.2.1! You have 52.49GB memory available.



bartMachine now using 3 cores.


In [9]:
nmin <- sum(training$status_group == 'functional needs repair')
nmin

[1] 3454

In [7]:
fits <- list()

In [14]:
#ctrl <- trainControl(method = 'cv', classProbs = TRUE, number = 1, repeats = 1)
#fits[[]] <- train(training[,predictors],
#                  factor(training[,'status_group.functional'], labels = c('F', 'T')),
#                        method = "bartMachine",
#                        use_missing_data = TRUE,
#                        tuneLength = 5,
#                        metric = "ROC",
#                        trControl = ctrl,
#                        ## Tell randomForest to sample by strata. Here, 
#                        ## that means within each class
#                       strata = factor(training[,'status_group.functional']),
#                        ## Now specify that the number of samples selected
#                        ## within each class should be the same
#                        sampsize = rep(nmin, 2))

In [15]:
?bartMachine


bartMachine {bartMachine},R Documentation
X,Data frame of predictors. Factors are automatically converted to dummies interally.
y,"Vector of response variable. If y is numeric or integer, a BART model for regression is built. If y is a factor with two levels, a BART model for classification is built."
Xy,A data frame of predictors and the response. The response column must be named “y”.
num_trees,The number of trees to be grown in the sum-of-trees model.
num_burn_in,Number of MCMC samples to be discarded as “burn-in”.
num_iterations_after_burn_in,Number of MCMC samples to draw from the posterior distribution of \hat{f}(x).
alpha,Base hyperparameter in tree prior for whether a node is nonterminal or not.
beta,Power hyperparameter in tree prior for whether a node is nonterminal or not.
k,"For regression, k determines the prior probability that E(Y|X) is contained in the interval (y_{min}, y_{max}), based on a normal distribution. For example, when k=2, the prior probability is 95%. For classification, k determines the prior probability that E(Y|X) is between (-3,3). Note that a larger value of k results in more shrinkage and a more conservative fit."
q,"Quantile of the prior on the error variance at which the data-based estimate is placed. Note that the larger the value of q, the more aggressive the fit as you are placing more prior weight on values lower than the data-based estimate. Not used for classification."


In [16]:
fits[['status_group.functional']] <- bartMachine(training[,predictors],
                                                 factor(training[,'status_group.functional']),
                                                 use_missing_data = TRUE, mem_cache_for_speed = FALSE)


bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 226 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for classification ...Covariate importance prior ON. Missing data feature ON. 
evaluating in sample data...done


In [19]:
fits[['status_group.non.functional']] <- bartMachine(training[,predictors],
                                                     factor(training[,'status_group.non.functional']),
                                                     use_missing_data = TRUE, mem_cache_for_speed = FALSE)


bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 226 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for classification ...Covariate importance prior ON. Missing data feature ON. 
evaluating in sample data...done


In [18]:
fits[['status_group.functional.needs.repair']] <- bartMachine(training[,predictors],
                                                              factor(training[,'status_group.functional.needs.repair']),
                                                              use_missing_data = TRUE, mem_cache_for_speed = FALSE)

bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 226 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for classification ...Covariate importance prior ON. Missing data feature ON. 
evaluating in sample data...done


In [57]:
fits[['status_group.functional.needs.repair balanced']] <- bartMachine(undersampled[,predictors],
                                                              factor(undersampled[,'status_group.functional.needs.repair']),
                                                              use_missing_data = TRUE, mem_cache_for_speed = FALSE)

bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 223 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for classification ...Covariate importance prior ON. Missing data feature ON. 
evaluating in sample data...done


In [58]:
levels(factor(undersampled[,'status_group.functional.needs.repair']))

[1] "0" "1"

In [59]:
fits

ERROR: Error in vapply(seq_along(mapped), function(i) {: values must be length 1,
 but FUN(X[[1]]) result is length 0


$status_group.functional
bartMachine v1.2.1 for classification

Missing data feature ON
training data n = 47522 and p = 225 
built in 3.53 mins on 3 cores, 50 trees, 250 burn-in and 1000 post. samples

confusion matrix:

           predicted 0 predicted 1 model errors
actual 0     15117.000    6597.000        0.304
actual 1      3055.000   22753.000        0.118
use errors       0.168       0.225        0.203


$status_group.functional.needs.repair
bartMachine v1.2.1 for classification

Missing data feature ON
training data n = 47522 and p = 225 
built in 3.78 mins on 3 cores, 50 trees, 250 burn-in and 1000 post. samples

confusion matrix:

           predicted 0 predicted 1 model errors
actual 0     43926.000     142.000        0.003
actual 1      3110.000     344.000        0.900
use errors       0.066       0.292        0.068


$status_group.non.functional
bartMachine v1.2.1 for classification

Missing data feature ON
training data n = 47522 and p = 225 
built in 3.47 mins on 3 core

In [129]:
names(fits)

[1] "status_group.functional"                      
[2] "status_group.functional.needs.repair"         
[3] "status_group.non.functional"                  
[4] "status_group.functional.needs.repair balanced"

### Functional

In [115]:
varImportance <- investigate_var_importance(fits[[1]], plot = FALSE)

.....


In [125]:
head(varImportance$avg_var_props, 10)

construction_year              longitude               latitude 
            0.05825629             0.05265388             0.05130359 
         date_recorded    date_recorded_month             population 
            0.03362847             0.02873581             0.02731731 
            amount_tsh date_recorded_year_ago     date_recorded_year 
            0.02473020             0.02421953             0.02247419 
     date_recorded_day 
            0.02022965

### Needs repair

In [120]:
varImportance2 <- investigate_var_importance(fits[[2]], plot = FALSE)

.....


In [126]:
head(varImportance2$avg_var_props, 10)

latitude              longitude      construction_year 
            0.05936618             0.05487109             0.04416318 
   date_recorded_month          date_recorded     date_recorded_year 
            0.04072795             0.03903473             0.02758806 
date_recorded_year_ago      date_recorded_day             population 
            0.01994181             0.01957125             0.01947912 
            amount_tsh 
            0.01431566

In [132]:
grep('date_recorded_year_.*', names(varImportance2$avg_var_props), value = TRUE)

[1] "date_recorded_year_ago"

### Non functional

In [123]:
varImportance3 <- investigate_var_importance(fits[[3]], plot = FALSE)

.....


In [127]:
head(varImportance3$avg_var_props, 10)

construction_year               latitude              longitude 
            0.05639995             0.04357914             0.04337736 
         date_recorded    date_recorded_month             population 
            0.03562667             0.02912006             0.02706897 
date_recorded_year_ago             amount_tsh     date_recorded_year 
            0.02359738             0.02327181             0.02121567 
     date_recorded_day 
            0.01623295

### Needs repair - balanced

In [ ]:
varImportance4 <- investigate_var_importance(fits[[4]], plot = FALSE)

In [128]:
head(varImportance4$avg_var_props, 10)

latitude          date_recorded              longitude 
            0.06526125             0.04804796             0.04776096 
     construction_year    date_recorded_month     date_recorded_year 
            0.04569747             0.03174621             0.02615305 
date_recorded_year_ago      date_recorded_day             amount_tsh 
            0.02569590             0.01966732             0.01350860 
       district_code_1 
            0.01338928

In [60]:
preds <- list()
for(outcome in c('status_group.functional', 'status_group.non.functional', 
                 'status_group.functional.needs.repair', 'status_group.functional.needs.repair balanced')) {
  preds[[outcome]] <- predict(fits[[outcome]], training[,predictors])
}

Warning message:
In pre_process_new_data(new_data, bart_machine): The following features were found in records for prediction which were not found in the original training data:
    region_code_40, scheme_management_None, extraction_type_other - mkulima/shinyanga
  These features will be ignored during prediction.

In [61]:
sapply(preds, head)

status_group.functional,status_group.non.functional,status_group.functional.needs.repair,status_group.functional.needs.repair balanced
0.978495270,0.008162931,0.012413910,0.128351265
0.801847786,0.219599288,0.009314847,0.265126798
0.7280951,0.1212882,0.1608879,0.5104547
0.935521044,0.157679508,0.006689232,0.120229342
0.53781501,0.42300008,0.01678798,0.13228723
0.555369574,0.580648887,0.006291532,0.301583529


In [23]:
for(outcome in c('status_group.functional', 'status_group.non.functional', 'status_group.functional.needs.repair')) {
    print(outcome)
    print(confusionMatrix(preds[[outcome]] > 0.5, training[,outcome] == 1))
}

[1] "status_group.functional"
Confusion Matrix and Statistics

          Reference
Prediction FALSE  TRUE
     FALSE 15117  3055
     TRUE   6597 22753
                                          
               Accuracy : 0.7969          
                 95% CI : (0.7932, 0.8005)
    No Information Rate : 0.5431          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5854          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.6962          
            Specificity : 0.8816          
         Pos Pred Value : 0.8319          
         Neg Pred Value : 0.7752          
             Prevalence : 0.4569          
         Detection Rate : 0.3181          
   Detection Prevalence : 0.3824          
      Balanced Accuracy : 0.7889          
                                          
       'Positive' Class : FALSE           
                               

In [62]:
predsTesting <- list()
for(outcome in c('status_group.functional', 'status_group.non.functional', 'status_group.functional.needs.repair',
                'status_group.functional.needs.repair balanced')) {
  predsTesting[[outcome]] <- predict(fits[[outcome]], testing[,predictors])
}
for(outcome in c('status_group.functional', 'status_group.non.functional', 'status_group.functional.needs.repair')) {
    print(outcome)
    print(confusionMatrix(predsTesting[[outcome]] > 0.5, testing[,outcome] == 1))
}

[1] "status_group.functional"
Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE  3708  783
     TRUE   1719 5668
                                          
               Accuracy : 0.7894          
                 95% CI : (0.7819, 0.7967)
    No Information Rate : 0.5431          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5697          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.6833          
            Specificity : 0.8786          
         Pos Pred Value : 0.8257          
         Neg Pred Value : 0.7673          
             Prevalence : 0.4569          
         Detection Rate : 0.3122          
   Detection Prevalence : 0.3781          
      Balanced Accuracy : 0.7809          
                                          
       'Positive' Class : FALSE           
                                  

In [73]:
predsDf <- data.frame('functional' = preds$status_group.functional,
                      'needrepair' = preds$`status_group.functional.needs.repair balanced`,
#                      'needrepair' = preds$status_group.functional.needs.repair,
                      'nonfunctional' = preds$status_group.non.functional)

In [74]:
head(predsDf)

,functional,needrepair,nonfunctional
1,0.9784953,0.1283513,0.008162931
2,0.8018478,0.2651268,0.2195993
3,0.7280951,0.5104547,0.1212882
4,0.935521,0.1202293,0.1576795
5,0.537815,0.1322872,0.4230001
6,0.5553696,0.3015835,0.5806489


In [76]:
library(dplyr)

colnames(predsDf) <- c('functional', 'functional needs repair', 'non functional')
predictionLevels <- factor(colnames(predsDf))
predictions <- predsDf %>%
                  rowwise() %>% 
                  mutate(prediction=predictionLevels[which.max(c(functional, `functional needs repair`, `non functional`))])

head(predictions)

,functional,functional needs repair,non functional,prediction
1,0.9784953,0.1283513,0.008162931,functional
2,0.8018478,0.2651268,0.2195993,functional
3,0.7280951,0.5104547,0.1212882,functional
4,0.935521,0.1202293,0.1576795,functional
5,0.537815,0.1322872,0.4230001,functional
6,0.5553696,0.3015835,0.5806489,non functional


In [77]:
table(predictions$prediction)


             functional functional needs repair          non functional 
                  25801                    8546                   13175 

In [88]:
length(predictions$prediction)
length(training$status_group)
predsForVote <- cbind(predictions, training$status_group)
colnames(predsForVote) <- c('functional', 'functional needs repair', 'non functional', 'prediction', 'status_group')
predsForVote$status_group.functional.needs.repair <- factor(training[,'status_group.functional.needs.repair'])
head(predsForVote)

[1] 47522

[1] 47522

,functional,functional needs repair,non functional,prediction,status_group,status_group.functional.needs.repair
1,0.9784953,0.1283513,0.008162931,functional,functional,0
2,0.8018478,0.2651268,0.2195993,functional,functional,0
3,0.7280951,0.5104547,0.1212882,functional,functional,0
4,0.935521,0.1202293,0.1576795,functional,functional,0
5,0.537815,0.1322872,0.4230001,functional,non functional,0
6,0.5553696,0.3015835,0.5806489,non functional,non functional,0


In [106]:
fitglm <- glm(status_group.functional.needs.repair ~  functional + `functional needs repair` + `non functional`, predsForVote,
             family = binomial)
#?predict.glm

In [101]:
summary(fitglm)


Call:
glm(formula = status_group.functional.needs.repair ~ functional + 
    `functional needs repair` + `non functional`, family = binomial, 
    data = predsForVote)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9932  -0.3435  -0.1993  -0.1361   3.3100  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -0.08233    0.22149  -0.372     0.71    
functional                -4.93877    0.19535 -25.282   <2e-16 ***
`functional needs repair`  3.73641    0.12757  29.289   <2e-16 ***
`non functional`          -5.16327    0.21019 -24.565   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 24761  on 47521  degrees of freedom
Residual deviance: 18368  on 47518  degrees of freedom
AIC: 18376

Number of Fisher Scoring iterations: 6


In [79]:
library(rpart)
fitrpart <- rpart(status_group ~  functional + `functional needs repair` + `non functional`, predsForVote,
                 control = rpart.control(cp = 0.001))

In [80]:
summary(fitrpart)

Call:
rpart(formula = status_group ~ functional + `functional needs repair` + 
    `non functional`, data = predsForVote, control = rpart.control(cp = 0.001))
  n= 47522 

           CP nsplit rel error    xerror        xstd
1 0.490236714      0 1.0000000 1.0000000 0.005001035
2 0.004305978      1 0.5097633 0.5099014 0.004243993
3 0.002555955      4 0.4929539 0.4943815 0.004198180
4 0.001335544      6 0.4878419 0.4914341 0.004189286
5 0.001000000      7 0.4865064 0.4896841 0.004183976

Variable importance
         non functional              functional functional needs repair 
                     48                      41                      11 

Node number 1: 47522 observations,    complexity param=0.4902367
  predicted class=functional               expected loss=0.4569252  P(node) =1
    class counts: 25808  3454 18260
   probabilities: 0.543 0.073 0.384 
  left son=2 (32421 obs) right son=3 (15101 obs)
  Primary splits:
      non functional          < 0.4608375  to the left,  i

In [81]:
predsForVoteRpart <- predsForVote
predsForVoteRpart$prediction <- predict(fitrpart, predsForVoteRpart, type = 'class')
head(predsForVoteRpart)

,functional,functional needs repair,non functional,prediction,status_group
1,0.9784953,0.1283513,0.008162931,functional,functional
2,0.8018478,0.2651268,0.2195993,functional,functional
3,0.7280951,0.5104547,0.1212882,functional,functional
4,0.935521,0.1202293,0.1576795,functional,functional
5,0.537815,0.1322872,0.4230001,functional,non functional
6,0.5553696,0.3015835,0.5806489,non functional,non functional


In [112]:
predsForVoteGlm <- predsForVote
predsForVoteGlm$prediction <- predict(fitglm, predsForVoteGlm, type = 'response') > 0.5
head(predsForVoteGlm)
table(predsForVoteGlm$prediction, predsForVoteGlm$status_group == 'functional needs repair')

,functional,functional needs repair,non functional,prediction,status_group,status_group.functional.needs.repair
1,0.9784953,0.1283513,0.008162931,FALSE,functional,0
2,0.8018478,0.2651268,0.2195993,FALSE,functional,0
3,0.7280951,0.5104547,0.1212882,FALSE,functional,0
4,0.935521,0.1202293,0.1576795,FALSE,functional,0
5,0.537815,0.1322872,0.4230001,FALSE,non functional,0
6,0.5553696,0.3015835,0.5806489,FALSE,non functional,0


       
        FALSE  TRUE
  FALSE 43707  2869
  TRUE    361   585

In [82]:
confusionMatrix(predictions$prediction, training$status_group)

Confusion Matrix and Statistics

                         Reference
Prediction                functional functional needs repair non functional
  functional                   20503                    1035           4263
  functional needs repair       3921                    2165           2460
  non functional                1384                     254          11537

Overall Statistics
                                          
               Accuracy : 0.7198          
                 95% CI : (0.7157, 0.7238)
    No Information Rate : 0.5431          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.5214          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: functional Class: functional needs repair
Sensitivity                     0.7944                        0.62681
Specificity                     0.7560                        0.85520
Pos Pred Value                 

In [83]:
confusionMatrix(predsForVoteRpart$prediction, training$status_group)

Confusion Matrix and Statistics

                         Reference
Prediction                functional functional needs repair non functional
  functional                   22790                    1981           4574
  functional needs repair        295                     689            207
  non functional                2723                     784          13479

Overall Statistics
                                          
               Accuracy : 0.7777          
                 95% CI : (0.7739, 0.7814)
    No Information Rate : 0.5431          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.577           
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: functional Class: functional needs repair
Sensitivity                     0.8831                        0.19948
Specificity                     0.6981                        0.98861
Pos Pred Value                 

In [84]:
predsTestingDf <- data.frame('functional' = predsTesting$status_group.functional,
                      'needrepair' = predsTesting$`status_group.functional.needs.repair balanced`,
                      'nonfunctional' = predsTesting$status_group.non.functional)

colnames(predsTestingDf) <- c('functional', 'functional needs repair', 'non functional')
predictionsTesting <- predsTestingDf %>%
                  rowwise() %>% 
                  mutate(prediction=predictionLevels[which.max(c(functional, `functional needs repair`, `non functional`))])

head(predictionsTesting)
predsForVoteTest <- cbind(predictionsTesting, testing$status_group)
colnames(predsForVoteTest) <- c('functional', 'functional needs repair', 'non functional', 'prediction', 'status_group')
head(predsForVoteTest)

predsForVoteRpart.test <- predsForVoteTest
predsForVoteRpart.test$prediction <- predict(fitrpart, predsForVoteRpart.test, type = 'class')
head(predsForVoteRpart.test)

,functional,functional needs repair,non functional,prediction
1,0.6750547,0.5143671,0.1702349,functional
2,0.0001385644,0.1306848,0.9909738,non functional
3,0.8970092,0.1046726,0.06849791,functional
4,0.947248,0.1014085,0.02019313,functional
5,0.5186912,0.5318944,0.3386338,functional needs repair
6,0.7460559,0.6597764,0.02066555,functional


,functional,functional needs repair,non functional,prediction,status_group
1,0.6750547,0.5143671,0.1702349,functional,functional
2,0.0001385644,0.1306848,0.9909738,non functional,non functional
3,0.8970092,0.1046726,0.06849791,functional,non functional
4,0.947248,0.1014085,0.02019313,functional,functional
5,0.5186912,0.5318944,0.3386338,functional needs repair,functional
6,0.7460559,0.6597764,0.02066555,functional,functional


,functional,functional needs repair,non functional,prediction,status_group
1,0.6750547,0.5143671,0.1702349,functional,functional
2,0.0001385644,0.1306848,0.9909738,non functional,non functional
3,0.8970092,0.1046726,0.06849791,functional,non functional
4,0.947248,0.1014085,0.02019313,functional,functional
5,0.5186912,0.5318944,0.3386338,functional,functional
6,0.7460559,0.6597764,0.02066555,functional,functional


In [85]:
confusionMatrix(predsForVoteRpart.test$prediction, testing$status_group)

Confusion Matrix and Statistics

                         Reference
Prediction                functional functional needs repair non functional
  functional                    5660                     518           1193
  functional needs repair         54                     154             45
  non functional                 737                     191           3326

Overall Statistics
                                         
               Accuracy : 0.7695         
                 95% CI : (0.7618, 0.777)
    No Information Rate : 0.5431         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.5599         
 Mcnemar's Test P-Value : < 2.2e-16      

Statistics by Class:

                     Class: functional Class: functional needs repair
Sensitivity                     0.8774                        0.17845
Specificity                     0.6847                        0.99101
Pos Pred Value                  0.7679 